In this version,
- Testing pipline of the selection mechanism





In [ ]:
import tensorflow as tf
print("Tensorflow version: ",tf.__version__)

In [ ]:
# Checking if cuda is there.
print("Cuda Availability: ", tf.test.is_built_with_cuda())

# Checking GPU is available or not.
print("GPU  Availability: ", tf.test.is_gpu_available())

# Check nos of GPUS
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Data Downloading

In [3]:
# choose the dataset version and paths
# 'modelnet40v1' or 'modelnet40v2' or 'shaded_modelnet40v2'
#(here modelnet40v1)
##########################
dataset_version= 'modelnet40v1' 
dataset_test= './data/modelnet40v1_test'

#dataset_version= 'modelnet40v2'
#dataset_test = './data/modelnet40v2_test'

#dataset_version= 'shaded_modelnet40v2'
#dataset_test = './data/modelnet40v2_test'

In [ ]:
# spicify the img size (here 224*224)
Img_Size= 224 

In [ ]:
# spicify selection mechanism
# the Most Similar View (MSV) or Most Dissimilar View (MDV)
# (here 'MSV')
#####################################

selection_mechanism = 'MSV'
#selection_mechanism = 'MDV'

# Data Processing


We've included helper functions that will label images, convert them into arrays, and then finally into a generator that will enable them to be loaded into the model in batches

In [4]:
# for training and testing
##########################

import random #Python Random module is an in-built module of Python which is used to generate random numbers.
from random import shuffle #is an inbuilt method of the random module. It is used to shuffle a sequence (list). Shuffling a list of objects means changing the position of the elements of the sequence
import cv2 # OpenCV-Python is a library of Python bindings designed to solve computer vision problems.
import os # Python OS module provides the facility to establish the interaction between the user and the operating system. It offers many useful OS functions that are used to perform OS-based tasks and get related information about operating system.
from tqdm import tqdm # is a library in Python which is used for creating Progress Meters or Progress Bars. tqdm got its name from the Arabic name taqaddum which means 'progress'
import matplotlib.pyplot as plt #Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python. Matplotlib makes easy things easy and hard things possible. Create publication quality plots. Make interactive figures that can zoom, pan, update. Customize visual style and layout.
import numpy as np #NumPy (Numerical Python) is an open-source library for the Python programming language. It is used for scientific computing and working with arrays.
from tensorflow.python.keras.utils.data_utils import Sequence #Keras and TensorFlow are open source Python libraries for working with neural networks, creating machine learning models and performing deep learning
# keras.utils This package provides utilities for Keras, such as modified callbacks, genereators, etc.
import tensorflow as tf


In [5]:
# for training and testing
##########################

#For shuffling or any thing random
import random #Python Random module is an in-built module of Python which is used to generate random numbers.
from random import shuffle #is an inbuilt method of the random module. It is used to shuffle a sequence (list). Shuffling a list of objects means changing the position of the elements of the sequence
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [6]:
# for training and testing
##########################

global num_views

if dataset_version == 'modelnet40v1' or dataset_version == 'original_modelnet40v1':
    num_views = 12
elif dataset_version == 'modelnet40v2':
    num_views = 20

print ("number of views is " + str(num_views))

number of views is 20


In [7]:
# for training and testing
##########################


def extract_View_No(img): # function to extract the view no
  ## Helper for process_data
  ViewNo =img[-6:-4] #take the last 2 characters from the img name before the ".png"
  if ViewNo == '01': return 1
  elif ViewNo == '02': return 2
  elif ViewNo == '03': return 3
  elif ViewNo == '04': return 4
  elif ViewNo == '05': return 5
  elif ViewNo == '06': return 6
  elif ViewNo == '07': return 7
  elif ViewNo == '08': return 8
  elif ViewNo == '09': return 9
  elif ViewNo == '10': return 10
  elif ViewNo == '11': return 11
  elif ViewNo == '12': return 12
  elif ViewNo == '13': return 13
  elif ViewNo == '14': return 14
  elif ViewNo == '15': return 15
  elif ViewNo == '16': return 16
  elif ViewNo == '17': return 17
  elif ViewNo == '18': return 18
  elif ViewNo == '19': return 19
  elif ViewNo == '20': return 20
  #print(ViewNo)


def label_image_encoder(img): # function to label image
  ## Helper for process_data
  label = img.split('_')[0] # from the image name, take the part before "_" and consider it as label #split() method is a beneficial tool for manipulating strings. It returns a list of strings after the main string is separated by a delimiter.
  if label == 'airplane': return 0 # encode airplane label as 0
  elif label == 'bathtub': return 1 # encode bathtub label as 1
  elif label == 'bed': return 2 # encode bed label as 2
  elif label == 'bench': return 3 # encode bench label as 3
  elif label == 'bookshelf': return 4 # encode bookshelf label as 4
  elif label == 'bottle': return 5 # encode bottle label as 5
  elif label == 'bowl': return 6 # encode bowl label as 6
  elif label == 'car': return 7 # encode car label as 7
  elif label == 'chair': return 8 # encode chair label as 8
  elif label == 'cone': return 9 # encode cone label as 9
  elif label == 'cup': return 10 # encode cup label as 10
  elif label == 'curtain': return 11 # encode curtain label as 11
  elif label == 'desk': return 12 # encode desk label as 12
  elif label == 'door': return 13 # encode door label as 13
  elif label == 'dresser': return 14 # encode dresser label as 14
  elif label == 'flower': return 15 # encode floer_pot label as 15
  elif label == 'glass': return 16 # encode glass_box label as 16
  elif label == 'guitar': return 17 # encode guitar label as 17
  elif label == 'keyboard': return 18 # encode keyboard label as 18
  elif label == 'lamp': return 19 # encode lamp label as 19
  elif label == 'laptop': return 20 # encode laptop label as 20
  elif label == 'mantel': return 21 # encode mantel label as 21
  elif label == 'monitor': return 22 # encode monitor label as 22
  elif label == 'night': return 23 # encode night_stand label as 23
  elif label == 'person': return 24 # encode person label as 24
  elif label == 'piano': return 25 # encode piano label as 25
  elif label == 'plant': return 26 # encode plant label as 26
  elif label == 'radio': return 27 # encode radio label as 27
  elif label == 'range': return 28 # encode range_hood label as 28
  elif label == 'sink': return 29 # encode sink label as 29
  elif label == 'sofa': return 30 # encode sofa label as 30
  elif label == 'stairs': return 31 # encode stairs label as 31
  elif label == 'stool': return 32 # encode stool label as 32
  elif label == 'table': return 33 # encode table label as 33
  elif label == 'tent': return 34 # encode tent label as 34
  elif label == 'toilet': return 35 # encode toilet label as 35
  elif label == 'tv': return 36 # encode tv_stand label as 36
  elif label == 'vase': return 37 # encode vase label as 37
  elif label == 'wardrobe': return 38 # encode wardrobe label as 38
  elif label == 'xbox': return 39 # encode xbox label as 39


def process_data(image_list, DATA_FOLDER, IMG_SIZE): # function that take the images, the data folder name, and the wanted image size, and retun the list of images as arrays with there labels and paths
  ## Helper for manual_pre_process
  ## Creates an array of images, labels, and file path
  data_df = [] #[] is a list: A multible collection of values, here define new array
  for img in tqdm(image_list): # tqdm . It will display a progress bar of the for loop, for each image in image_list do the following
    path = os.path.join(DATA_FOLDER, img) # concatenates various path components with exactly one directory separator ('/'), concatenate DATA_FOLDER path then / then the image name to create an image's path
    ViewNo = extract_View_No(img) # extract the view no
    label = label_image_encoder(img) # use the above function to encode the image's label
    img = cv2.imread(path, cv2.IMREAD_COLOR) # cv2. imread() method loads an image from the specified file. IMREAD_COLOR reads the image with RGB colors but no transparency channel, load image from its path
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) # Resizing the image img, by only change the width=IMG_SIZE and height=IMG_SIZE of the image, this step to make all image with the same size
    #data_df.append([np.array(img), np.array(label), path]) #Append in Python is a pre-defined method used to add a single item to certain collection types. so, append the list data_df by adding sum-list with 3 element for each image [np.array(img)=image as array, np.array(label)=image's label as 0 or 1, path=image's path]it
    data_df.append([np.array(img), np.array(label), ViewNo, path]) #Append in Python is a pre-defined method used to add a single item to certain collection types. so, append the list data_df by adding sum-list with 3 element for each image [np.array(img)=image as array, np.array(label)=image's label as 0 or 1, path=image's path]it
  
  return data_df # return the data_df list after process it


def manual_pre_process(dir, IMG_SIZE):
  '''
  Creates an array of images, labels, and files from a directory of image files

  Args:
    dir: string, folder name
    IMG_SIZE: int, image height and width

  Returns
    X: (n x IMG_SIZE x IMG_SIZE) numpy array of images
    y: (n,) numpy array of labels
    files: (n,) numpy array of files

  '''
  image_lst = sorted(os.listdir(dir)) # get the list of all files and directories in the specified directory "dir"
  data_df = process_data(image_lst, dir, IMG_SIZE) # take the images, the data folder name, and the wanted image size, and retun the list of images after processing as arrays with there labels and paths
  X = np.array([i[0] for i in data_df]).reshape(-1, IMG_SIZE, IMG_SIZE, 3) # take the images only (as arrays) from data_df and save them as X (the input images)
  y = np.array([i[1] for i in data_df]) # take the labels only from data_df and save them as Y (the actual labels)
  ViewNos = np.array([i[2] for i in data_df]) # take the ViewNo only from data_df and save them as ViewNo
  files = np.array([i[3] for i in data_df]) # take the images' paths only from data_df and save them as files
  return X, y, ViewNos, files # return X=the images(as arrays), y=the actual labels, and files=the images' paths
  


In [8]:
# for training and testing
##########################

# Dictionary
class_info = {0: 'airplane',
              1: 'bathtub',
              2: 'bed',
              3: 'bench',
              4: 'bookshelf',
              5: 'bottle',
              6: 'bowl',
              7: 'car',
              8: 'chair',
              9: 'cone',
              10: 'cup',
              11: 'curtain',
              12: 'desk',
              13: 'door',
              14: 'dresser',
              15: 'flower_pot',
              16: 'glass_box',
              17: 'guitar',
              18: 'keyboard',
              19: 'lamp',
              20: 'laptop',
              21: 'mantel',
              22: 'monitor',
              23: 'night_stand',
              24: 'person',
              25: 'piano',
              26: 'plant',
              27: 'radio',
              28: 'range_hood',
              29: 'sink',
              30: 'sofa',
              31: 'stairs',
              32: 'stool',
              33: 'table',
              34: 'tent',
              35: 'toilet',
              36: 'tv_stand',
              37: 'vase',
              38: 'wardrobe',
              39: 'xbox'} # define a dictionary (a key/value mapping) of the labels


#X, y, files = manual_pre_process(val_imgs, 224) # call the manual_pre_process function by passing the directory of the dataset and the wanted image size 224*224

#X, y, ViewNo, files = manual_pre_process(modelnet40v1_train, 224)


#X_train, y_train, ViewNo_train, files_train = manual_pre_process(modelnet40v1_train_subset, 224)
X_test, y_test, ViewNo_test, files_test = manual_pre_process(dataset_test, Img_Size)


100%|██████████| 49360/49360 [01:32<00:00, 534.10it/s] 


In [24]:
#for testing
########################

test_NoOfSamples, h, w, channel= X_test.shape

print("Number of testing Samples is")
print(test_NoOfSamples)

Number of testing Samples is
49360


In [ ]:
# for testing
##########################
# Note: spicify the subplot rows and columns based on the number of views
##########################


# show the (12 or 20 views) of one object samples before feature extraction
plt.figure(figsize=(15, 15)) # create a fig of size 15*15

for i in range(0,num_views):
 img = X_test[i] #take sample image
 label = y_test[i] #take the label of the sample image
 viewNo= ViewNo_test[i]
 path = files_test[i] #take the path of the sample image

 ax = plt.subplot(4, 5, i +1) # for 20 views
 #sax = plt.subplot(3, 4, i +1) # for 12 views
 plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) # show the img in the plot
 plt.axis('off') # remove the axis
 category = class_info[label.item()] # convert the label from ndarray to int then bring its class
 title= category.capitalize() + ', View No:' + str(viewNo)
 plt.title(title) # show the title above the img

# Training the Pre-trained Model (Stage 1)

In [11]:
# for training and testing
##########################

from keras.utils.np_utils import to_categorical
categorical_y_test = to_categorical(y_test, 40)

In [13]:
# for training and testing
##########################

# Loading the Pre-Trained Model
# Upload the required libraries

from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.applications.inception_v3 import InceptionV3, preprocess_input
import time
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from tqdm import tqdm

# keras imports
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import DenseNet169
from keras.applications.densenet import DenseNet201
from keras.applications.nasnet import NASNetLarge
from keras.applications.nasnet import NASNetMobile
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.resnet import ResNet50
from keras.applications.resnet import ResNet101
from keras.applications.resnet import ResNet152
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.resnet_v2 import ResNet101V2
from keras.applications.resnet_v2 import ResNet152V2

# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.efficientnet import EfficientNetB0, preprocess_input
from keras.applications.efficientnet import EfficientNetB1, preprocess_input
from keras.applications.efficientnet import EfficientNetB2, preprocess_input
from keras.applications.efficientnet import EfficientNetB3, preprocess_input
from keras.applications.efficientnet import EfficientNetB4, preprocess_input
from keras.applications.efficientnet import EfficientNetB5, preprocess_input
from keras.applications.efficientnet import EfficientNetB6, preprocess_input
from keras.applications.efficientnet import EfficientNetB7, preprocess_input

from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input
from keras.layers import Dense,GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
from pycm import *
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from numpy import array
from sklearn.model_selection import KFold

from tqdm import tqdm_notebook as tqdm
from keras_tqdm import TQDMNotebookCallback
from keras_tqdm import TQDMCallback
#from keras_preprocessing.image import load_img
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import gradient_descent_v2
from keras.optimizers import adam_v2
from keras.layers import MaxPooling2D

import keras
from tensorflow import keras
from keras import optimizers
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from keras.layers import Flatten
from tensorflow.keras.layers import ( # from tensorflow.keras.layers 
     Flatten, Dropout, Dense #,BatchNormalization, SeparableConv2D, Activation, 
)
from tensorflow.keras.utils import plot_model ## from tensorflow.keras.utils 
from PIL import Image

In [14]:
# for training and testing
##########################


def plot_acc_loss(history, PLOT_NAME, model_name_txt): 
    fig = plt.figure(figsize=(10,5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    #plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    #plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.savefig("./Results/"+str(dataset_version)+"/"+str(model_name_txt)+"/"+str(PLOT_NAME))
    plt.show()

def getPrediction(preds): # preds is the otput scors from softmax
  im_class= np.argmax(preds, axis = 0) # the index as an array of one element , argmax Returns the indices of the maximum values along an axis
  #"the index of the class"
  idx= im_class
  #"the probability is"
  propability= preds[idx]
  # the class label
  label= class_info[idx]
  return idx, label, propability

In [ ]:
#all_deep_models = [VGG16, VGG19, MobileNet, InceptionV3, InceptionResNetV2, Xception, DenseNet121, DenseNet169, DenseNet201, NASNetMobile, MobileNetV2, ResNet50, ResNet101, ResNet152, ResNet50V2, ResNet101V2, ResNet152V2, NASNetLarge ]
#all_model_name_txt = ["VGG16", "VGG19", "MobileNet", "InceptionV3", "InceptionResNetV2", "Xception", "DenseNet121", "DenseNet169", "DenseNet201", "NASNetMobile", "MobileNetV2",  "ResNet50", "ResNet101", "ResNet152", "ResNet50V2", "ResNet101V2", "ResNet152V2", "NASNetLarge"]


In [15]:
# for training and testing
##########################
# Note: spicify the deep model name
##########################


all_deep_models = [ResNet152]
all_model_name_txt = ["ResNet152"]


In [16]:
# for training and testing
##########################
# Note: spicify the BATCH_SIZE based on the number of views
#       spicify the img size that expected by the deep model
##########################


CLASSES = 40
WIDTH = Img_Size
HEIGHT = Img_Size
BATCH_SIZE = 384 #20 shapes with a total of 400 views (20*20 =400) for 20-views version,  and 32 shapes with a total of 384 views (32*12 =384)) for 12-views version
EPOCHS = 20 
learning_rate = 0.0001

In [17]:
from tensorflow.keras.utils import Sequence
import numpy as np   

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

In [19]:
from tensorflow.keras.utils import Sequence
import numpy as np   

class DataGenerator1(Sequence):
    def __init__(self, x_set, batch_size):
        self.x = x_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x

In [20]:
# for testing
##########################

i=0
for deep_model in all_deep_models:
    model_name = deep_model
    model_name_txt = str(all_model_name_txt[i])
    MODEL_FILE = "./Results/"+str(dataset_version)+"/"+str(model_name_txt)+"/"+str(model_name_txt)+"_stage1.model"
    model_1 = load_model(MODEL_FILE)
    i+=1

## Feature Extraction

In [25]:
# extract and save the features incrementally as baches and clear the memory occupied by the previous batches.

from tensorflow.keras.models import Model
from tensorflow.keras import backend as K


# Constants
batch_size = 100
output_directory = "./data/"

# Calculate the number of batches
num_images = test_NoOfSamples
num_batches = int(np.ceil(num_images / batch_size))

# Initialize an empty list to store features
all_features = []

# Iterate through each batch
#for i in range(1500, 1969):
for i in range(num_batches):
    start_index = i * batch_size
    end_index = min((i + 1) * batch_size, num_images)

    # Initialize an empty list to store features in the current batch
    batch_features = []

    # Save images in the current batch as features
    for j in range(start_index, end_index):
        batch_features.append(X_test[j])

    # Convert batch features list to a NumPy array
    batch_features = np.array(batch_features)

    # Extract features of the the current batch using the model
    batch_features = model_1.predict(batch_features, verbose=1) 

    # Append batch features to the list of all features
    all_features.append(batch_features)

    # Clear session to free up GPU memory
    K.clear_session() # a K.clear_session() call after each batch to clear the TensorFlow session and free up GPU memory. 

    print(f"Batch {i+1}/{num_batches} processed.")

    # Save features after every 10 batches (adjust as needed)
    if (i+1) % 10 == 0 or (i+1) == num_batches:
        # Concatenate all batches of features into a single NumPy array
        all_features = np.concatenate(all_features)

        # Save features to a separate file
        output_file = f"./data/features_batch_{i+1}.npy"
        np.save(output_file, all_features) # This allows you to save the features incrementally and clear the memory occupied by the previous batches.
        # Clear the list of features to save memory
        all_features = []

print("Feature extraction completed.")

4/4 [==============================] - 7s 206ms/step
Batch 1/494 processed.
4/4 [==============================] - 1s 136ms/step
Batch 2/494 processed.
4/4 [==============================] - 1s 137ms/step
Batch 3/494 processed.
4/4 [==============================] - 1s 136ms/step
Batch 4/494 processed.
4/4 [==============================] - 1s 138ms/step
Batch 5/494 processed.
4/4 [==============================] - 1s 142ms/step
Batch 6/494 processed.
4/4 [==============================] - 1s 135ms/step
Batch 7/494 processed.
4/4 [==============================] - 1s 138ms/step
Batch 8/494 processed.
4/4 [==============================] - 1s 139ms/step
Batch 9/494 processed.
4/4 [==============================] - 1s 138ms/step
Batch 10/494 processed.
4/4 [==============================] - 1s 141ms/step
Batch 11/494 processed.
4/4 [==============================] - 1s 139ms/step
Batch 12/494 processed.
4/4 [==============================] - 1s 146ms/step
Batch 13/494 processed.
4/4 [===

In [26]:
# Grouping training features into a single NumPy array.

import numpy as np

# Calculate the number of batches
batch_size=100
num_images = test_NoOfSamples
num_batches = int(np.ceil(num_images / batch_size))
#num_batches = 1969  # Total number of batches
batch_interval = 10  # Interval between batches

# Initialize an empty list to store features
all_testing_features = []

# Iterate through each batch
for i in range(1, num_batches + 1):
    # Check if the current batch is within the desired interval
    if i % batch_interval == 0:
        # Load features from the file
        features_file = f"./data/features_batch_{i}.npy"
        batch_features = np.load(features_file).astype(np.float16)

        # Append batch features to the list of all features
        all_testing_features.append(batch_features)
        print("features_batch_",str(i)," is appended")

        if i == 490: # the batch before the last batch
            features_file = f"./data/features_batch_494.npy"
            batch_features = np.load(features_file)
            # Append batch features to the list of all features
            all_testing_features.append(batch_features)
            print("features_batch_494 is appended")

    

# Concatenate all batches of features into a single NumPy array
all_testing_features = np.concatenate(all_testing_features)

# Save all features to a single NumPy array file
output_file = "./data/all_testing_features.npy"
np.save(output_file, all_testing_features)

print("All testing features are grouped into a single NumPy array.")

# The objects must then be expanded from a 3D array to a 4D array with the dimensions [samples, rows, cols, channels]
print("Shape:", all_testing_features.shape) # VGG16 Shape: (49360, 7, 7, 2048)
print("Data type:", all_testing_features.dtype)

features_batch_ 10  is appended
features_batch_ 20  is appended
features_batch_ 30  is appended
features_batch_ 40  is appended
features_batch_ 50  is appended
features_batch_ 60  is appended
features_batch_ 70  is appended
features_batch_ 80  is appended
features_batch_ 90  is appended
features_batch_ 100  is appended
features_batch_ 110  is appended
features_batch_ 120  is appended
features_batch_ 130  is appended
features_batch_ 140  is appended
features_batch_ 150  is appended
features_batch_ 160  is appended
features_batch_ 170  is appended
features_batch_ 180  is appended
features_batch_ 190  is appended
features_batch_ 200  is appended
features_batch_ 210  is appended
features_batch_ 220  is appended
features_batch_ 230  is appended
features_batch_ 240  is appended
features_batch_ 250  is appended
features_batch_ 260  is appended
features_batch_ 270  is appended
features_batch_ 280  is appended
features_batch_ 290  is appended
features_batch_ 300  is appended
features_batch_ 310

In [ ]:
# Load all features to a single NumPy array file

features_file = "./data/all_testing_features.npy"
all_testing_features = np.load(features_file)

print("All testing features are load into a single NumPy array.")

# Verify the shape and data type of the loaded array
print("Shape:", all_testing_features.shape) # ResNet50 Shape: (49360, 7, 7, 2048)
print("Data type:", all_testing_features.dtype)

# Convert views to feature vectors

https://towardsdatascience.com/image-similarity-detection-in-action-with-tensorflow-2-0-b8d9a78b2509

An image feature vector is a list of numbers that represents a whole image, typically used for image similarity calculations or image classification tasks.


## Feature vectors using flatten


In [27]:
# convert to flatten feature vector
########################

import numpy as np

# Flatten each element in the array
flattened_arr = all_testing_features.reshape(all_testing_features.shape[0], -1)

# Update the original array with the flattened elements
all_testing_features = flattened_arr

del flattened_arr

# Print the shape of the updated array
print(all_testing_features.shape) #(49360, 100352)
print(type(all_testing_features))

(49360, 100352)
<class 'numpy.ndarray'>


# View Selection: Importance Score

In [28]:
# for training and testing
##########################


# Define Similarity Metrics

import math
from math import sqrt

# Define Cosine Similarity
def cosineSim(a1,a2): # after avoid division by zero
    summ = 0
    suma1 = 0
    sumb1 = 0
    for i,j in zip(a1, a2):
        suma1 += i * i
        sumb1 += j*j
        summ += i*j
        eps= 0.00000001 #Small value to avoid division by zero. Default: 1e-8
    denominator = (sqrt(suma1))*(sqrt(sumb1))
    MaxDenominator = max(denominator, eps)
    cosine_sim = summ / MaxDenominator
    #cosine_sim = 1 - cosine_sim # to clalculate the cosine distance
    #CS_SCORE= 1-CS_SCORE # DISTANCE SIMILARITY
    # CS_SCORE= 1-CS_SCORE # to measure the cosine distance
    return cosine_sim

# Define jaccard Similarity
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

def average(x):
    assert len(x) > 0
    return float(sum(x)) / len(x)

# Define pearson Similarity
def pearson_def(x, y):
    assert len(x) == len(y)
    n = len(x)
    assert n > 0
    avg_x = average(x)
    avg_y = average(y)
    diffprod = 0
    xdiff2 = 0
    ydiff2 = 0
    for idx in range(n):
        xdiff = x[idx] - avg_x
        ydiff = y[idx] - avg_y
        diffprod += xdiff * ydiff
        xdiff2 += xdiff * xdiff
        ydiff2 += ydiff * ydiff

    return diffprod / math.sqrt(xdiff2 * ydiff2)

In [29]:
# for training and testing
##########################
# Note: spicify the fig size based on the number of views
##########################


def Plot_with_Important_Scores(im_ls, GuidedGradCAM, IS, classes, ViewNo, n):
    """

    """
    #random.shuffle(im_ls)
    #plt.subplots(figsize=(30, 10*n))
    #k=1
    for i in range(n):
        plt.figure(figsize=(15, 15))
        k=1
        #img = cv2.imread(path[i])
        img= im_ls[i]
        Guided_img= GuidedGradCAM[i]
        Important_Score= IS[i]

        # Show original image
        plt.subplot(3,4,k) # for 12 view
        plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
        label = classes[i] #take the label of the sample image
        viewNo= ViewNo[i]
        category = class_info[label.item()] # convert the label from ndarray to int then bring its class
        #print(category)
        title= category.capitalize() + ', View No:' + str(viewNo)
        plt.title(title, fontsize=10)
        plt.axis("off")

        # Show guided GradCAM
        plt.subplot(3,4,k+1)
        plt.imshow(cv2.cvtColor(Guided_img,cv2.COLOR_BGR2RGB))
        #title= "Guided GradCAM" + ', I\u0302\u2082'+ str(viewNo)+':'+ str(Important_Score)
        title= 'Important Score:'+ str(Important_Score)
        plt.title(title, fontsize=10)
        plt.axis("off")
        plt.show()
        #k += 2


In [30]:
# for testing
##########################

# Applay View Selection
#######################
#######################

from sklearn import preprocessing

start_index= 0
test_object_label=0
end_index= num_views 

n= len(all_testing_features)
object_views=np.zeros(num_views)

test_NoOfObject= test_NoOfSamples//num_views
NoOfPixels= WIDTH *HEIGHT*3

X_test_global_descriptor=[]
y_test_global_descriptor=[]
IS_test_global_descriptor=[]
viewNo_test_global_descriptor=[]


finished = "false"
while finished != "true":
# make smaller array conataining only the FVs of the 12 views of one object
# by slicing the test_array to take the n views each times
  object_views= all_testing_features[start_index:end_index]
  test_object_label= y_test[start_index]

# Calculate importance scores
#############################
  k=0 # counter for the view
  j= 0 # counter for the other comparied to view
  test_importance_Scores=np.zeros(num_views)
# compute the importance scores of the 12 views of one object
  for k in range(num_views): # k is the view that I want to calculate its score
    View_Score=0
    for j in range(num_views):
      CS_SCORE= cosineSim(object_views[k],object_views[j]) # CS_SCORE is the Cosine Similarity
      View_Score= View_Score + CS_SCORE
    test_importance_Scores[k]=View_Score

# Normalize the importance scores
#################################
  test_normalized_importance_Scores= [float(i)/sum(test_importance_Scores) for i in test_importance_Scores] #to normalize against the sum to ensure that the sum is always 1.0 (or as close to as possible).

  #if start_index == 0:
    #Plot_with_Important_Scores(X_test, X_test_GudiedGradCAM, test_normalized_importance_Scores, y_test, ViewNo_test, n=num_views)


# Compute the global descriptor of an object (the most similar view which has the maximum important score\Cosine Similarity )
###############################################################################
  
  arr = np.array(test_normalized_importance_Scores) #convert test_normalized_importance_Scores from list to array
  
  if selection_mechanism == 'MSV':
    view_pos = np.argmax(arr) # find the index of the view that contain the  maximum important score\Cosine Similarity (that most similar to other views)
  elif selection_mechanism == 'MDV':
    view_pos = np.argmin(arr) # find the index of the view that contain the  minimum important score\Cosine Similarity (that most dissimilar to other views)

  IS_test_global_descriptor.append(test_normalized_importance_Scores[view_pos]) # save the maximum or minimum important score\Cosine Similarity
  X_test_global_descriptor.append(object_views[view_pos]) # take the view that contain the maximum or minimum important score\Cosine Similarity (that most similar to other views) as a global_descriptor to the object
  y_test_global_descriptor.append(test_object_label) # take the label of the view that has the maximum or minimum important score\Cosine Similarity
  viewNo_test_global_descriptor.append(view_pos+1)

# Increase indexes to go to the next object
  start_index+= num_views
  end_index+= num_views

  if (end_index > n): # go through all images and break if it exceed the array index
    finished = "true"


In [34]:
# for testing
##########################

# convert the new labels to categorical labels
from tensorflow.keras.utils import to_categorical
categorical_y_test_global_descriptor = to_categorical(y_test_global_descriptor, 40)

In [32]:
# for training and testing
##########################

# to print the full NumPy array, without truncation

import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [35]:
# for testing
##########################

# convert the training data from (list) to (nparray)
y_test_global_descriptor = np.array(y_test_global_descriptor)
X_test_global_descriptor= np.array(X_test_global_descriptor)

# ensure that both are ndarray
print(type(X_test_global_descriptor))
print(type(y_test_global_descriptor))
print(type(categorical_y_test_global_descriptor))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


## Do the testing:

In [36]:
# for testing
##########################
# extract the nunmber of tested objects (samples) and thier number of features (features)

print(X_test_global_descriptor.shape)
samples= X_test_global_descriptor.shape[0] #2468
features= X_test_global_descriptor.shape[1] # rows* cols* channels = 7*7*2048 =100352

(2468, 100352)


In [37]:
# for testing
##########################

# Reshape the feature vectors into a 2D matrix

# we have a list of samples precomputed feature vectors (of samples objects),
# each with a length of features
X_test_global_descriptor = np.array(X_test_global_descriptor).reshape(samples, features)
print(X_test_global_descriptor.shape)

(2468, 100352)


In [38]:
# Save all features and labels to a single NumPy array file
output_file = "./data/all_testing_labels_global_descriptor.npy"
np.save(output_file, y_test_global_descriptor)

output_file = "./data/all_testing_global_descriptor.npy"
np.save(output_file, X_test_global_descriptor)

# Prediction and Results calculation:

In [39]:
# for testing
##########################
# Note: spicify the classes if using the modelnet10
##########################

from sklearn.metrics import confusion_matrix
from operator import truediv
import matplotlib.pylab as plt
#import numpy as np


def plot_confusion_matrix_manual(cm, cmap=plt.cm.Blues):
    classes = ['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair',
                   'cone', 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot', 'glass_box',
                   'guitar', 'keyboard', 'lamp', 'laptop', 'mantel', 'monitor', 'night_stand',
                   'person', 'piano', 'plant', 'radio', 'range_hood', 'sink', 'sofa', 'stairs',
                   'stool', 'table', 'tent', 'toilet', 'tv_stand', 'vase', 'wardrobe', 'xbox']
    plt.rcParams['savefig.dpi'] = 300
    plt.rcParams['figure.dpi'] = 300
    
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    plt.tick_params(labelsize=5)
    # We want to show all ticks...
    ax.set(
        xticks=np.arange(cm.shape[1]),
        yticks=np.arange(cm.shape[0]),
        # ... and label them with the respective list entries
        xticklabels=classes,
        yticklabels=classes)
    plt.title('Confusion matrix', fontsize=6)
    plt.xlabel('Predicted label', fontsize=6)
    plt.ylabel('True label', fontsize=6)
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(),
             rotation=45,
             ha="right",
             rotation_mode="anchor")

In [40]:
# for training and testing
##########################
# to print the full NumPy array, without truncation

import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

## Test FCL: 20epochs

In [41]:
# for testing
########################## Load FCL: 20epochs ##############################

i=0
for deep_model in all_deep_models:
    model_name = deep_model
    model_name_txt = str(all_model_name_txt[i])
    # Load the fully-connected Layer (FCL)
    MODEL_FILE = "./Results/"+str(dataset_version)+"/"+str(model_name_txt)+"/"+"20epochs"+"/"+str(model_name_txt)+"_stage2_FCL.model"
    model_2 = load_model(MODEL_FILE)
    model_2.summary()
    i+=1

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 40)                4014120   
Total params: 4,014,120
Trainable params: 4,014,120
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# for testing
########################## Test FCL: 20epochs ##############################

#import seaborn as sn
from mlxtend.plotting import plot_confusion_matrix

# Do the prediction
predictions = model_2.predict(X_test_global_descriptor) # predict the classe of the test images
(samples, classes)= predictions.shape

# Extract the index of prediction from the preds_scores_matrix
y_pred= np.zeros_like(y_test_global_descriptor)
y_pred_classes= np.empty(len(y_test_global_descriptor), dtype=object)

for i in range(samples):
   preds_scores_matrix = predictions[i, :]
   idx, label, propability = getPrediction(preds_scores_matrix)
   y_pred[i]= idx
   y_pred_classes[i] = label



# return the predicted classe of the test images as labels such as "airplane
y_true_classes = np.empty(len(y_test_global_descriptor), dtype=object)

for t in range (len(y_test_global_descriptor)):
   y_true_classes[t] = class_info[y_test_global_descriptor[t]]



#Confution Matrix and Classification Report
print('Confusion Matrix')
print(confusion_matrix(y_true_classes, y_pred_classes))



print('Classification Report')

target_names = ['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair', 'cone',
                'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot', 'glass_box', 'guitar', 'keyboard', 'lamp',
                'laptop', 'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant', 'radio', 'range_hood', 'sink',
                'sofa', 'stairs','stool', 'table', 'tent', 'toilet', 'tv_stand', 'vase', 'wardrobe', 'xbox']

print(classification_report(y_true_classes, y_pred_classes, target_names=target_names))
print ("--- y_true_classes")
print (y_true_classes)
print ("--- y_pred_classes")
print (y_pred_classes)
ccm = confusion_matrix(y_true_classes, y_pred_classes) # Create ConfusionMatrix From Data

results = "./Results/"+str(dataset_version)+"/"+str(model_name_txt)+"/"+str(model_name_txt)+"_20epochs_FCL_Results.txt"
f = open(results, "w")
cm = ConfusionMatrix(actual_vector=y_true_classes, predict_vector=y_pred_classes) # Create ConfusionMatrix From categorical Data

disp = ConfusionMatrixDisplay(confusion_matrix=ccm, display_labels=target_names)
disp.plot(cmap=plt.cm.Blues, xticks_rotation=90)
plt.tight_layout()
plt.savefig("./Results/"+str(dataset_version)+"/"+str(model_name_txt)+"/"+"20epochs_FCL_ConfusionMatrix.png", pad_inches=5)
plt.tick_params(axis='both', which='major', labelsize=5)
plt.show()

f.write("{}\n".format(cm))
f.close()

In [43]:
print(classification_report(y_true_classes, y_pred_classes, target_names=target_names))

              precision    recall  f1-score   support

    airplane       0.99      1.00      1.00       100
     bathtub       0.88      0.58      0.70        50
         bed       0.87      0.90      0.89       100
       bench       0.57      0.80      0.67        20
   bookshelf       0.79      0.85      0.82       100
      bottle       0.91      0.93      0.92       100
        bowl       0.74      0.85      0.79        20
         car       0.98      0.99      0.99       100
       chair       0.82      0.92      0.87       100
        cone       0.90      0.95      0.93        20
         cup       0.58      0.55      0.56        20
     curtain       0.86      0.95      0.90        20
        desk       0.73      0.72      0.73        86
        door       0.83      0.95      0.88        20
     dresser       0.83      0.84      0.83        86
  flower_pot       0.36      0.60      0.45        20
   glass_box       0.76      0.74      0.75       100
      guitar       0.98    

In [44]:
from sklearn.metrics import accuracy_score

# Obtain the true labels and predicted labels
true_labels = y_true_classes  # Replace with your true labels
predicted_labels = y_pred_classes # Replace with your predicted labels

# Calculate overall accuracy
overall_accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate average accuracy
class_labels = set(true_labels)  # Get the unique class labels
num_classes = len(class_labels)
class_accuracies = []

for label in class_labels:
    # Filter the true and predicted labels corresponding to the current class
    true_labels_class = [true_labels[i] for i in range(len(true_labels)) if true_labels[i] == label]
    predicted_labels_class = [predicted_labels[i] for i in range(len(predicted_labels)) if true_labels[i] == label]
    
    # Calculate accuracy for the current class
    class_accuracy = accuracy_score(true_labels_class, predicted_labels_class)
    class_accuracies.append(class_accuracy)

average_accuracy = sum(class_accuracies) / num_classes

# Print overall accuracy and average accuracy
print("Class Accuracy:")
for i in range(40):
    print(target_names[i], "=", class_accuracies[i])
print("------------------------------------------")
print("------------------------------------------")
print("------------------------------------------")
print("Overall Accuracy:", overall_accuracy*100, "%")
print("Average Accuracy:", average_accuracy*100, "%")

Class Accuracy:
airplane = 0.99
bathtub = 0.75
bed = 0.93
bench = 0.55
bookshelf = 0.91
bottle = 0.6
bowl = 0.6
car = 0.58
chair = 0.98
cone = 0.74
cup = 0.55
curtain = 0.93
desk = 0.45
door = 0.6
dresser = 0.68
flower_pot = 0.7209302325581395
glass_box = 0.95
guitar = 0.7
keyboard = 0.85
lamp = 0.5
laptop = 0.92
mantel = 0.79
monitor = 0.8
night_stand = 0.86
person = 0.95
piano = 0.93
plant = 0.71
radio = 0.83
range_hood = 0.9
sink = 1.0
sofa = 1.0
stairs = 0.65
stool = 0.85
table = 0.686046511627907
tent = 0.59
toilet = 0.95
tv_stand = 0.8
vase = 0.8372093023255814
wardrobe = 0.95
xbox = 0.6
------------------------------------------
------------------------------------------
------------------------------------------
Overall Accuracy: 81.52350081037277 %
Average Accuracy: 77.91046511627906 %


## Test FCL: 30epochs

In [45]:
# for testing
########################## Load FCL: 30epochs ##############################

i=0
for deep_model in all_deep_models:
    model_name = deep_model
    model_name_txt = str(all_model_name_txt[i])
    # Load the fully-connected Layer (FCL)
    MODEL_FILE = "./Results/"+str(dataset_version)+"/"+str(model_name_txt)+"/"+"30epochs"+"/"+str(model_name_txt)+"_stage2_FCL.model"
    model_2 = load_model(MODEL_FILE)
    model_2.summary()
    i+=1

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 40)                4014120   
Total params: 4,014,120
Trainable params: 4,014,120
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# for testing
########################## Test FCL: 30epochs ##############################

#import seaborn as sn
from mlxtend.plotting import plot_confusion_matrix

# Do the prediction
predictions = model_2.predict(X_test_global_descriptor) # predict the classe of the test images
(samples, classes)= predictions.shape

# Extract the index of prediction from the preds_scores_matrix
y_pred= np.zeros_like(y_test_global_descriptor)
y_pred_classes= np.empty(len(y_test_global_descriptor), dtype=object)

for i in range(samples):
   preds_scores_matrix = predictions[i, :]
   idx, label, propability = getPrediction(preds_scores_matrix)
   y_pred[i]= idx
   y_pred_classes[i] = label



# return the predicted classe of the test images as labels such as "airplane
y_true_classes = np.empty(len(y_test_global_descriptor), dtype=object)

for t in range (len(y_test_global_descriptor)):
   y_true_classes[t] = class_info[y_test_global_descriptor[t]]


#Confution Matrix and Classification Report
print('Confusion Matrix')
print(confusion_matrix(y_true_classes, y_pred_classes))

print('Classification Report')
#####  I may remove the class number from the target_names
target_names = ['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair', 'cone',
                'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot', 'glass_box', 'guitar', 'keyboard', 'lamp',
                'laptop', 'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant', 'radio', 'range_hood', 'sink',
                'sofa', 'stairs','stool', 'table', 'tent', 'toilet', 'tv_stand', 'vase', 'wardrobe', 'xbox']

print(classification_report(y_true_classes, y_pred_classes, target_names=target_names))
print ("--- y_true_classes")
print (y_true_classes)
print ("--- y_pred_classes")
print (y_pred_classes)
ccm = confusion_matrix(y_true_classes, y_pred_classes) # Create ConfusionMatrix From Data

results = "./Results/"+str(dataset_version)+"/"+str(model_name_txt)+"/"+str(model_name_txt)+"_30epochs_FCL_Results.txt"
f = open(results, "w")
cm = ConfusionMatrix(actual_vector=y_true_classes, predict_vector=y_pred_classes) # Create ConfusionMatrix From categorical Data

disp = ConfusionMatrixDisplay(confusion_matrix=ccm, display_labels=target_names)
disp.plot(cmap=plt.cm.Blues, xticks_rotation=90)
plt.tight_layout()
plt.savefig("./Results/"+str(dataset_version)+"/"+str(model_name_txt)+"/"+"30epochs_FCL_ConfusionMatrix.png", pad_inches=5)
plt.tick_params(axis='both', which='major', labelsize=5)
plt.show()

f.write("{}\n".format(cm))
f.close()

In [47]:
print(classification_report(y_true_classes, y_pred_classes, target_names=target_names))

              precision    recall  f1-score   support

    airplane       0.99      1.00      1.00       100
     bathtub       0.92      0.68      0.78        50
         bed       0.88      0.92      0.90       100
       bench       0.57      0.85      0.68        20
   bookshelf       0.85      0.84      0.84       100
      bottle       0.90      0.95      0.93       100
        bowl       0.81      0.85      0.83        20
         car       0.98      0.99      0.99       100
       chair       0.86      0.92      0.89       100
        cone       0.90      0.95      0.93        20
         cup       0.61      0.55      0.58        20
     curtain       0.86      0.90      0.88        20
        desk       0.75      0.72      0.73        86
        door       0.83      0.95      0.88        20
     dresser       0.87      0.87      0.87        86
  flower_pot       0.34      0.60      0.44        20
   glass_box       0.81      0.76      0.78       100
      guitar       0.98    

In [48]:
from sklearn.metrics import accuracy_score

# Obtain the true labels and predicted labels
true_labels = y_true_classes  # Replace with your true labels
predicted_labels = y_pred_classes # Replace with your predicted labels

# Calculate overall accuracy
overall_accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate average accuracy
class_labels = set(true_labels)  # Get the unique class labels
num_classes = len(class_labels)
class_accuracies = []

for label in class_labels:
    # Filter the true and predicted labels corresponding to the current class
    true_labels_class = [true_labels[i] for i in range(len(true_labels)) if true_labels[i] == label]
    predicted_labels_class = [predicted_labels[i] for i in range(len(predicted_labels)) if true_labels[i] == label]
    
    # Calculate accuracy for the current class
    class_accuracy = accuracy_score(true_labels_class, predicted_labels_class)
    class_accuracies.append(class_accuracy)

average_accuracy = sum(class_accuracies) / num_classes

# Print overall accuracy and average accuracy
print("Class Accuracy:")
for i in range(40):
    print(target_names[i], "=", class_accuracies[i])
print("------------------------------------------")
print("------------------------------------------")
print("------------------------------------------")
print("Overall Accuracy:", overall_accuracy*100, "%")
print("Average Accuracy:", average_accuracy*100, "%")

Class Accuracy:
airplane = 0.99
bathtub = 0.8
bed = 0.95
bench = 0.55
bookshelf = 0.93
bottle = 0.65
bowl = 0.6
car = 0.68
chair = 0.98
cone = 0.76
cup = 0.6
curtain = 0.96
desk = 0.6
door = 0.6
dresser = 0.72
flower_pot = 0.7209302325581395
glass_box = 0.9
guitar = 0.7
keyboard = 0.85
lamp = 0.55
laptop = 0.92
mantel = 0.8
monitor = 0.95
night_stand = 0.87
person = 0.95
piano = 0.95
plant = 0.76
radio = 0.83
range_hood = 0.92
sink = 1.0
sofa = 1.0
stairs = 0.6
stool = 0.84
table = 0.6744186046511628
tent = 0.7
toilet = 0.95
tv_stand = 0.85
vase = 0.872093023255814
wardrobe = 0.95
xbox = 0.7
------------------------------------------
------------------------------------------
------------------------------------------
Overall Accuracy: 83.79254457050244 %
Average Accuracy: 80.44360465116279 %


## Test FCN: 20epochs

In [49]:
# for testing
########################## Load FCN: 20epochs ##############################

i=0
for deep_model in all_deep_models:
    model_name = deep_model
    model_name_txt = str(all_model_name_txt[i])
    # Load the fully-connected network (FCN)
    MODEL_FILE = "./Results/"+str(dataset_version)+"/"+str(model_name_txt)+"/"+"20epochs"+"/"+str(model_name_txt)+"_stage2_FCN.model"
    model_2 = load_model(MODEL_FILE)
    model_2.summary()
    i+=1

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              102761472 
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 40)                41000     
Total params: 102,802,472
Trainable params: 102,802,472
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# for testing
########################## Test FCN: 20epochs ##############################

#import seaborn as sn
from mlxtend.plotting import plot_confusion_matrix

# Do the prediction
predictions = model_2.predict(X_test_global_descriptor) # predict the classe of the test images
(samples, classes)= predictions.shape

# Extract the index of prediction from the preds_scores_matrix
y_pred= np.zeros_like(y_test_global_descriptor)
y_pred_classes= np.empty(len(y_test_global_descriptor), dtype=object)

for i in range(samples):
   preds_scores_matrix = predictions[i, :]
   idx, label, propability = getPrediction(preds_scores_matrix)
   y_pred[i]= idx
   y_pred_classes[i] = label



# return the predicted classe of the test images as labels such as "airplane
y_true_classes = np.empty(len(y_test_global_descriptor), dtype=object)

for t in range (len(y_test_global_descriptor)):
   y_true_classes[t] = class_info[y_test_global_descriptor[t]]
 
#Confution Matrix and Classification Report
print('Confusion Matrix')
print(confusion_matrix(y_true_classes, y_pred_classes))
#tn, fp, fn, tp = confusion_matrix(y_true_classes, y_pred_classes).ravel()



print('Classification Report')
target_names = ['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair', 'cone',
                'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot', 'glass_box', 'guitar', 'keyboard', 'lamp',
                'laptop', 'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant', 'radio', 'range_hood', 'sink',
                'sofa', 'stairs','stool', 'table', 'tent', 'toilet', 'tv_stand', 'vase', 'wardrobe', 'xbox']

print(classification_report(y_true_classes, y_pred_classes, target_names=target_names))
print ("--- y_true_classes")
print (y_true_classes)
print ("--- y_pred_classes")
print (y_pred_classes)
ccm = confusion_matrix(y_true_classes, y_pred_classes) # Create ConfusionMatrix From Data

results = "./Results/"+str(dataset_version)+"/"+str(model_name_txt)+"/"+str(model_name_txt)+"_20epochs_FCN_Results.txt"
f = open(results, "w")
cm = ConfusionMatrix(actual_vector=y_true_classes, predict_vector=y_pred_classes) # Create ConfusionMatrix From categorical Data

disp = ConfusionMatrixDisplay(confusion_matrix=ccm, display_labels=target_names)
disp.plot(cmap=plt.cm.Blues, xticks_rotation=90)
plt.tight_layout()
plt.savefig("./Results/"+str(dataset_version)+"/"+str(model_name_txt)+"/"+"20epochs_FCN_ConfusionMatrix.png", pad_inches=5)
plt.tick_params(axis='both', which='major', labelsize=5)
plt.show()

f.write("{}\n".format(cm))
f.close()

In [51]:
print(classification_report(y_true_classes, y_pred_classes, target_names=target_names))

              precision    recall  f1-score   support

    airplane       0.98      1.00      0.99       100
     bathtub       0.75      0.36      0.49        50
         bed       0.72      0.91      0.81       100
       bench       0.58      0.70      0.64        20
   bookshelf       0.70      0.86      0.77       100
      bottle       0.92      0.96      0.94       100
        bowl       0.81      0.85      0.83        20
         car       0.98      0.95      0.96       100
       chair       0.70      0.92      0.80       100
        cone       0.95      0.95      0.95        20
         cup       0.61      0.55      0.58        20
     curtain       0.89      0.80      0.84        20
        desk       0.65      0.60      0.63        86
        door       0.79      0.75      0.77        20
     dresser       0.68      0.76      0.72        86
  flower_pot       0.31      0.55      0.40        20
   glass_box       0.69      0.75      0.72       100
      guitar       0.99    

In [52]:
from sklearn.metrics import accuracy_score

# Obtain the true labels and predicted labels
true_labels = y_true_classes  # Replace with your true labels
predicted_labels = y_pred_classes # Replace with your predicted labels

# Calculate overall accuracy
overall_accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate average accuracy
class_labels = set(true_labels)  # Get the unique class labels
num_classes = len(class_labels)
class_accuracies = []

for label in class_labels:
    # Filter the true and predicted labels corresponding to the current class
    true_labels_class = [true_labels[i] for i in range(len(true_labels)) if true_labels[i] == label]
    predicted_labels_class = [predicted_labels[i] for i in range(len(predicted_labels)) if true_labels[i] == label]
    
    # Calculate accuracy for the current class
    class_accuracy = accuracy_score(true_labels_class, predicted_labels_class)
    class_accuracies.append(class_accuracy)

average_accuracy = sum(class_accuracies) / num_classes

# Print overall accuracy and average accuracy
print("Class Accuracy:")
for i in range(40):
    print(target_names[i], "=", class_accuracies[i])
print("------------------------------------------")
print("------------------------------------------")
print("------------------------------------------")
print("Overall Accuracy:", overall_accuracy*100, "%")
print("Average Accuracy:", average_accuracy*100, "%")

Class Accuracy:
airplane = 0.95
bathtub = 0.63
bed = 0.97
bench = 0.55
bookshelf = 0.93
bottle = 0.45
bowl = 0.55
car = 0.36
chair = 1.0
cone = 0.75
cup = 0.45
curtain = 0.94
desk = 0.05
door = 0.55
dresser = 0.61
flower_pot = 0.6046511627906976
glass_box = 0.8
guitar = 0.6
keyboard = 0.85
lamp = 0.15
laptop = 0.92
mantel = 0.78
monitor = 0.7
night_stand = 0.78
person = 0.85
piano = 0.96
plant = 0.67
radio = 0.8
range_hood = 0.91
sink = 1.0
sofa = 0.95
stairs = 0.55
stool = 0.86
table = 0.5348837209302325
tent = 0.51
toilet = 0.75
tv_stand = 0.7
vase = 0.7558139534883721
wardrobe = 0.95
xbox = 0.55
------------------------------------------
------------------------------------------
------------------------------------------
Overall Accuracy: 76.90437601296597 %
Average Accuracy: 70.56337209302326 %


## Test FCN: 30epochs

In [53]:
# for testing
########################## Load FCN: 30epochs ##############################

i=0
for deep_model in all_deep_models:
    model_name = deep_model
    model_name_txt = str(all_model_name_txt[i])
    # Load the fully-connected network (FCN)
    MODEL_FILE = "./Results/"+str(dataset_version)+"/"+str(model_name_txt)+"/"+"30epochs"+"/"+str(model_name_txt)+"_stage2_FCN.model"
    model_2 = load_model(MODEL_FILE)
    model_2.summary()
    i+=1

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              102761472 
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 40)                41000     
Total params: 102,802,472
Trainable params: 102,802,472
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# for testing
########################## Test FCN: 30epochs ##############################

#import seaborn as sn
from mlxtend.plotting import plot_confusion_matrix

# Do the prediction
predictions = model_2.predict(X_test_global_descriptor) # predict the classe of the test images
(samples, classes)= predictions.shape

# Extract the index of prediction from the preds_scores_matrix
y_pred= np.zeros_like(y_test_global_descriptor)
y_pred_classes= np.empty(len(y_test_global_descriptor), dtype=object)

for i in range(samples):
   preds_scores_matrix = predictions[i, :]
   idx, label, propability = getPrediction(preds_scores_matrix)
   y_pred[i]= idx
   y_pred_classes[i] = label



# return the predicted classe of the test images as labels such as "airplane
y_true_classes = np.empty(len(y_test_global_descriptor), dtype=object)

for t in range (len(y_test_global_descriptor)):
   y_true_classes[t] = class_info[y_test_global_descriptor[t]]


#Confution Matrix and Classification Report
print('Confusion Matrix')
print(confusion_matrix(y_true_classes, y_pred_classes))
#tn, fp, fn, tp = confusion_matrix(y_true_classes, y_pred_classes).ravel()



print('Classification Report')
#####  I may remove the class number from the target_names
target_names = ['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair', 'cone',
                'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot', 'glass_box', 'guitar', 'keyboard', 'lamp',
                'laptop', 'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant', 'radio', 'range_hood', 'sink',
                'sofa', 'stairs','stool', 'table', 'tent', 'toilet', 'tv_stand', 'vase', 'wardrobe', 'xbox']

print(classification_report(y_true_classes, y_pred_classes, target_names=target_names))
print ("--- y_true_classes")
print (y_true_classes)
print ("--- y_pred_classes")
print (y_pred_classes)
ccm = confusion_matrix(y_true_classes, y_pred_classes) # Create ConfusionMatrix From Data

results = "./Results/"+str(dataset_version)+"/"+str(model_name_txt)+"/"+str(model_name_txt)+"_30epochs_FCN_Results.txt"
f = open(results, "w")
cm = ConfusionMatrix(actual_vector=y_true_classes, predict_vector=y_pred_classes) # Create ConfusionMatrix From categorical Data

disp = ConfusionMatrixDisplay(confusion_matrix=ccm, display_labels=target_names)
disp.plot(cmap=plt.cm.Blues, xticks_rotation=90)
plt.tight_layout()
plt.savefig("./Results/"+str(dataset_version)+"/"+str(model_name_txt)+"/"+"30epochs_FCN_ConfusionMatrix.png", pad_inches=5)
plt.tick_params(axis='both', which='major', labelsize=5)
plt.show()

f.write("{}\n".format(cm))
f.close()

In [55]:
print(classification_report(y_true_classes, y_pred_classes, target_names=target_names))

              precision    recall  f1-score   support

    airplane       0.99      1.00      1.00       100
     bathtub       0.83      0.48      0.61        50
         bed       0.80      0.93      0.86       100
       bench       0.71      0.75      0.73        20
   bookshelf       0.74      0.87      0.80       100
      bottle       0.92      0.97      0.95       100
        bowl       0.74      0.85      0.79        20
         car       0.97      0.99      0.98       100
       chair       0.79      0.92      0.85       100
        cone       1.00      1.00      1.00        20
         cup       0.63      0.60      0.62        20
     curtain       0.86      0.90      0.88        20
        desk       0.69      0.69      0.69        86
        door       0.86      0.95      0.90        20
     dresser       0.77      0.77      0.77        86
  flower_pot       0.31      0.55      0.39        20
   glass_box       0.76      0.76      0.76       100
      guitar       1.00    

In [56]:
from sklearn.metrics import accuracy_score

# Obtain the true labels and predicted labels
true_labels = y_true_classes  # Replace with your true labels
predicted_labels = y_pred_classes # Replace with your predicted labels

# Calculate overall accuracy
overall_accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate average accuracy
class_labels = set(true_labels)  # Get the unique class labels
num_classes = len(class_labels)
class_accuracies = []

for label in class_labels:
    # Filter the true and predicted labels corresponding to the current class
    true_labels_class = [true_labels[i] for i in range(len(true_labels)) if true_labels[i] == label]
    predicted_labels_class = [predicted_labels[i] for i in range(len(predicted_labels)) if true_labels[i] == label]
    
    # Calculate accuracy for the current class
    class_accuracy = accuracy_score(true_labels_class, predicted_labels_class)
    class_accuracies.append(class_accuracy)

average_accuracy = sum(class_accuracies) / num_classes

# Print overall accuracy and average accuracy
print("Class Accuracy:")
for i in range(40):
    print(target_names[i], "=", class_accuracies[i])
print("------------------------------------------")
print("------------------------------------------")
print("------------------------------------------")
print("Overall Accuracy:", overall_accuracy*100, "%")
print("Average Accuracy:", average_accuracy*100, "%")

Class Accuracy:
airplane = 0.99
bathtub = 0.7
bed = 0.98
bench = 0.6
bookshelf = 0.93
bottle = 0.5
bowl = 0.65
car = 0.48
chair = 1.0
cone = 0.76
cup = 0.55
curtain = 0.95
desk = 0.15
door = 0.55
dresser = 0.73
flower_pot = 0.686046511627907
glass_box = 0.9
guitar = 0.65
keyboard = 0.85
lamp = 0.3
laptop = 0.92
mantel = 0.77
monitor = 0.85
night_stand = 0.81
person = 1.0
piano = 0.97
plant = 0.75
radio = 0.81
range_hood = 0.93
sink = 1.0
sofa = 1.0
stairs = 0.65
stool = 0.87
table = 0.6046511627906976
tent = 0.61
toilet = 0.95
tv_stand = 0.75
vase = 0.7674418604651163
wardrobe = 1.0
xbox = 0.65
------------------------------------------
------------------------------------------
------------------------------------------
Overall Accuracy: 81.03727714748784 %
Average Accuracy: 76.42034883720929 %
